# Contents
##1. Collect Data
##2. Prepare dataset (input-output - 256x256 each)
##3. Clone Pix2Pix Repo
##4. Train Model
##5. Export Model
##6. Convert Model to tfjs

#1. Collect Data

I used the **ffhq dataset** available at https://github.com/NVlabs/ffhq-dataset

It is available under [Creative Commons BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license

**Paper Citation:**

>A Style-Based Generator Architecture for Generative Adversarial Networks
Tero Karras (NVIDIA), Samuli Laine (NVIDIA), Timo Aila (NVIDIA)
https://arxiv.org/abs/1812.04948

#2. Prepare Dataset

## 2.1 Upload and unzip data

In [ ]:
!cp drive/MyDrive/colorization/38000.zip ./

In [ ]:
!unzip 38000.zip

## 2.2 Resize data

In [ ]:
import cv2

In [ ]:
import os

In [ ]:
all_images = [image_name for image_name in os.listdir("./38000")]

In [ ]:
os.mkdir("dataset")

In [ ]:
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
for image_name in tqdm(all_images):
  img = cv2.imread("./38000/%s"%image_name)
  img = cv2.resize(img, (256, 256))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
  img_dataset = np.hstack([img_gray, img])
  cv2.imwrite("./dataset/%s"%image_name, img_dataset)

100%|██████████| 1000/1000 [00:08<00:00, 118.08it/s]


# 3. Clone Pix2Pix Repo

## Huge thanks to https://github.com/yining1023/pix2pix_tensorflowjs_lite and https://github.com/affinelayer/pix2pix-tensorflow repos for the detailed explanation

In [ ]:
!git clone https://github.com/affinelayer/pix2pix-tensorflow

Cloning into 'pix2pix-tensorflow'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 24.86 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [ ]:
%cd pix2pix-tensorflow/

/content/pix2pix-tensorflow


In [ ]:
%%shell
mkdir -p photos/combined

In [ ]:
%%shell
cp ../dataset/* photos/combined/

#4. Train model

##4.1 Split Dataset

In [ ]:
%%shell
python tools/split.py --dir photos/combined

##4.2 Train Model

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
## Replace 'import tensorflow as tf' with these two lines in pix2pix.py if you face any errors with the next line

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
%%shell
python pix2pix.py --mode train --output_dir colorized_train --max_epochs 200 --input_dir photos/combined/train --which_direction AtoB --ngf 32 --ndf 32

In [ ]:
%%shell
python pix2pix.py --mode test --output_dir pikachu_test --input_dir photos/combined/val --checkpoint colorized_train

#5. Export checkpoints

In [ ]:
%%shell
python pix2pix.py --mode export --output_dir ./export/ --checkpoint ./colorized_train/ --which_direction AtoB

Instructions for updating:
non-resource variables are not supported in the long term
loaded lab_colorization = False
loaded ndf = 32
loaded ngf = 32
loaded which_direction = AtoB
aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = ./colorized_train/
display_freq = 0
flip = False
gan_weight = 1.0
input_dir = None
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = None
max_steps = None
mode = export
ndf = 32
ngf = 32
output_dir = ./export/
output_filetype = png
progress_freq = 50
save_freq = 5000
scale_size = 256
seed = 707106167
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:536: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `l

#6. Convert to Tensorflow JS model

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
## Replace 'import tensorflow as tf' with these two lines in server/tools/dump_checkpoints/tensorflow_checkpoint_dumper.py if you face any errors with the next line

In [ ]:
%%shell
cd server
mkdir -p ./static/models
python tools/export-checkpoint.py --checkpoint ../export --output_file ./static/models/colorized_AtoB.pict

Instructions for updating:
non-resource variables are not supported in the long term
Writing variable generator/encoder_8/conv2d/bias...
Writing variable generator/encoder_8/batch_normalization/moving_variance...
Writing variable generator/encoder_8/batch_normalization/moving_mean...
Writing variable generator/encoder_8/batch_normalization/gamma...
Writing variable generator/encoder_8/batch_normalization/beta...
Writing variable generator/encoder_7/conv2d/kernel...
Writing variable generator/encoder_7/conv2d/bias...
Writing variable generator/encoder_7/batch_normalization/moving_variance...
Writing variable generator/decoder_4/conv2d_transpose/kernel...
Writing variable generator/decoder_7/batch_normalization/moving_mean...
Writing variable generator/encoder_6/conv2d/bias...
Writing variable generator/decoder_7/batch_normalization/gamma...
Writing variable generator/decoder_6/batch_normalization/moving_mean...
Writing variable generator/decoder_7/batch_normalization/beta...
Writing var

In [ ]:
%%shell
cp server/static/models/colorized_AtoB.pict /content/drive/MyDrive/colorization/